In [50]:
import random
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

In [69]:
def generate_balanced_brackets(n, min_depth, max_depth):
    sequence = ''
    stack = []
    
    # for i in range(min_depth):
    #     sequence += '('
    #     stack.append('(')
    start = random.randint(0, n - 2*min_depth)
    # print(start)
    
    for i in range(n):
        if i > start and i < start + min_depth and max_depth > len(stack):
            sequence += '('
            stack.append('(')
            continue
        open_or_close = random.choice([0, 1]) and stack
        left = n - i
        # print(left, n, i, min_depth)
        if left < len(stack) or (not open_or_close and len(stack) == left) or len(stack) == max_depth:
            open_or_close = 1
        if open_or_close:
            sequence += ')'
            stack.pop()
        else:
            sequence += '('
            stack.append('(')
    
    return sequence

def is_balanced(s):
    stack = []
    max_depth = 0
    count = 0
    
    for bracket in s:
        if bracket == ')':
            if stack:
                stack.pop()
            else:
                stack.append(bracket)
                break
        else:
            stack.append(bracket)
            max_depth = max(max_depth, len(stack))
    
    for bracket in s:
        if bracket == '(':
            count += 1
        else:
            count -= 1
                
    if stack:
        return -1 * max_depth, count
    return max_depth, count

def generate_bracket_sequence(n, depth):
    brackets = ['(', ')']
    start = random.randint(0, n - depth)
    max_depth = 0
    stack = []
    
    sequence = '('
    for i in range(n-1):
        if i > start and i < start + depth:
            sequence += '('
            stack.append('(')
            continue
        if len(stack) < depth:
            sequence += random.choice(brackets)
            stack.append(sequence[-1])
        else:
            sequence += ')'
            stack.pop()
        if sequence[-1] == '(':
            stack.append('(')
        max_depth = max(max_depth, len(stack))
    if is_balanced(sequence)[1] == 0:
        return sequence
    else:
        return sequence + random.choice(brackets)
    # seq = generate_balanced_brackets(n, depth, depth)
    # bal = is_balanced(seq)
    # while bal[0] != -1*depth:
    #     seq2 = seq
    #     index = random.randint(0, n-1)
    #     if seq[index] == '(':
    #         seq2 = seq[:index] + ')' + seq[index+1:]
    #     else:
    #         seq2 = seq[:index] + '(' + seq[index+1:]
    #     bal = is_balanced(seq2)
    # return seq2


In [88]:
# for _ in range(100):
s = generate_bracket_sequence(100, 5)
    # if is_balanced(s)[0] != -5:
print(is_balanced(s), s)

(-2, -53) (()()())))())))())))))))))))))())))())())))))()))(((()))))))())))())())())())))))())))())())()))))))(


In [89]:
s = generate_balanced_brackets(100, 1, 1)
print(is_balanced(s), s)

(1, 0) ()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()()


In [90]:

balanced_df = pd.DataFrame(columns=['sequence', 'stack_depth', 'count'])

seqs = [0 for i in range(400)]

generated_seqs = set()

gen = [i for i in range(2, 10)] + [i for i in range(10, 101, 10)]
print(f'Generating sequences in the following stack depths: {gen}')

for j in tqdm(gen):
    while True:
        length = 512
        
        sequence = generate_balanced_brackets(length, j, j)
        balanced, count = is_balanced(sequence)
        
        if seqs[balanced] == 1000 or sequence in generated_seqs or balanced not in gen:
            continue
            
        seqs[balanced] += 1
        
        balanced_df.loc[len(balanced_df)] = [sequence, balanced, count]
        generated_seqs.add(sequence)
        
        if seqs[j] >= 1000:
            break
    # print(seqs)

Generating sequences in the following stack depths: [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


  0%|          | 0/18 [00:00<?, ?it/s]

In [91]:
print(len(balanced_df))
balanced_df.head()

18000


,sequence,stack_depth,count
0,()(()())()(()()())(())()()()(())()(())(())(())...,2,0
1,()()()(())(()()())(())(())()(())(()()())(())()...,2,0
2,()(())(())(()()())()(()())(()())(())(()())()()...,2,0
3,(())()()(())(())()()()()()(()())(()())()()()()...,2,0
4,()(()())(()()()()())(())(())()()()()(())(()())...,2,0


In [93]:

unbalanced_data = []
for index, i in tqdm(enumerate(gen), total=len(gen)):
    while len(unbalanced_data) < 1000*(index+1):
        sequence = generate_bracket_sequence(512, i)
        balanced, count = is_balanced(sequence)
        # if balanced != -1*i:
            # print(i, end=' ')
            # continue
        unbalanced_data.append(sequence)
        
    # print(len(unbalanced_data))

print(len(unbalanced_data))
unbalanced_stack_depth = [ is_balanced(sequence) for sequence in unbalanced_data ]

unbalanced_count = [x[1] for x in unbalanced_stack_depth]
unbalanced_stack_depth = [x[0] for x in unbalanced_stack_depth]

  0%|          | 0/18 [00:00<?, ?it/s]

18000


In [94]:

unbalanced_df = pd.DataFrame({'sequence': unbalanced_data, 'stack_depth': unbalanced_stack_depth, 'count': unbalanced_count}, columns=['sequence', 'stack_depth', 'count'])

In [95]:
bracket_data = pd.concat([balanced_df, unbalanced_df], ignore_index=True)

bracket_data = bracket_data.sample(frac=1).reset_index(drop=True)

In [96]:
bracket_data

,sequence,stack_depth,count
0,((()()))(((()()(()()))(())())())))()))())))))(...,-5,-221
1,(()(((()()(())(((())())))))())((()(((()))())((...,9,0
2,(())(())()()()()()(((()))(()(()))(()()(()()()(...,4,0
3,()())()()()())))((((())))())(((()()()(()((((()...,-1,-175
4,())()((((()))()))())())))))())))())))))())))()...,-1,-273
...,...,...,...
35995,(()())())))())())))())))())))())())))())())())...,-2,-299
35996,()()()()(())(()()())()()(())()()()()()()(()())...,2,0
35997,()()((()())()(())()(())(())())()(()(()()()())(...,3,0
35998,((())))))())())))())))())))())())))())))()))))...,-3,-303


In [97]:
# find minimum stack depth
min_stack_depth = bracket_data['stack_depth'].min()

min_stack_depth

-122

In [98]:
max_stack_depth = bracket_data['stack_depth'].max()

max_stack_depth

100

In [99]:
bracket_data.to_csv('balanced_brackets.csv', index=False)

In [20]:
def generate_no_count(n):
    sequence = ['(' for i in range(n//2)] + [')' for i in range(n//2)]
    sequence = ''.join(sequence)
    # print(sequence, is_balanced(sequence))
    while is_balanced(sequence)[0] > 0:
        # randomly shuffle the sequence
        sequence = ''.join(random.sample(sequence, len(sequence)))
    return sequence

In [48]:
s = generate_no_count(10)
is_balanced(s), s

((-1, 0), '())())()((')

In [104]:
no_count_data = []
for i in tqdm(range(4000)):
    sequence = generate_no_count(512)
    no_count_data.append(sequence)
    
no_count_stack_depth = [ is_balanced(sequence) for sequence in no_count_data ]

no_count_count = [x[1] for x in no_count_stack_depth]
no_count_stack_depth = [x[0] for x in no_count_stack_depth]

no_count_df = pd.DataFrame({'sequence': no_count_data, 'stack_depth': no_count_stack_depth, 'count': no_count_count}, columns=['sequence', 'stack_depth', 'count'])

  0%|          | 0/4000 [00:00<?, ?it/s]

In [105]:
no_count_df.head()

,sequence,stack_depth,count
0,))()((((()((((())())((())()((()()())(())))()((...,0,0
1,))(()))(()))(())(((()()((()))(())(()(())(()(((...,0,0
2,)(((()(((()())())((()()(((()()()))())(()(())((...,0,0
3,()))((()())(())))))))(((()(()))())))()(())((()...,-1,0
4,)((()))))()))))((())(()())(())((())()))()))(()...,0,0


In [102]:
no_count_df.to_csv('no_count.csv', index=False)

In [106]:
# append bracket data and no count data
combined_df = pd.concat([bracket_data, no_count_df], ignore_index=True)

combined_df = combined_df.sample(frac=1).reset_index(drop=True)

combined_df.to_csv('balanced_with_no_count.csv', index=False)